# Part II: Modeling
---

[>>>Link to the presentation<<<](https://docs.google.com/presentation/d/1TooN8ycOvl-FB0gWikqjoO2LvPg2Vlh-09dW1wUhXB8/edit?usp=sharing)

In this section, the pre-processed datasets will be vectorized (converted into numbers) by a DictVectorizer so that computers will be able to understand tham
1. Data Preparation
2. Model Comparison
- K-mean
- LDA
- NMF
- Top2Vec > still in progress and will continue if time permits
- BERTopic > give up BERTopic because of time constraints

## 0. Import Libraries

In [76]:
# Import Basic Libraries
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Import Sklearn Libraries
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans
from sklearn import model_selection, datasets
from sklearn.tree import DecisionTreeClassifier
import joblib
import pickle

# Import NLP Libraries
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary
from pythainlp.tokenize import sent_tokenize, word_tokenize
from pythainlp.corpus import thai_stopwords
import re
from tqdm import tqdm 
import pyLDAvis
import pyLDAvis.gensim_models
from top2vec import Top2Vec

# Set default Thai font
mpl.font_manager.fontManager.addfont('./THSarabunNew/THSarabunNew.ttf')
mpl.rc('font', family='TH Sarabun New', size=20)


# Set random seed
np.random.seed(42)

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

## 1. Data Preparation

### 1.1 Load Datasets

In [33]:
# Load datasets
bkkbiz = pd.read_json('../datasets/bkkbiz_digital_processed.json')
matichon = pd.read_json('../datasets/matichon_digital_processed.json')

In [34]:
# Check shape
bkkbiz.shape

(16464, 13)

In [35]:
# Check head
bkkbiz.head()

,ID,date,category,title,description,article,url,title_count,title_tokenize,description_count,description_tokenize,article_count,article_tokenize
0,620015,2014-11-27 10:08:00,การเงิน-การลงทุน,เนชั่นจ่อปรับค่าโฆษณา20%,"""เนชั่น มัลติมีเดีย กรุ๊ป""หวังรายได้ปีหน้าแตะ3...",นางสาวดวงกมล โชตะนา กรรมการผู้อำนวยการ บริษัท ...,https://www.bangkokbiznews.com/news/detail/620015,6,"[เนชั่น, จ่อ, ค่าโฆษณา]",25,"[เนชั่น, มัลติมีเดีย, กรุ๊ป, หวัง, รายได้, ปีห...",738,"[ดวงกมล, โช, ตะ, นา, กรรมการ, ผู้อำนวยการ, บริ..."
1,620023,2014-11-27 10:26:00,การเมือง,ทุ่ม80ล้านเปิดเวที77จว.,สปช.ลุยฟังความเห็นปฏิรูป-คิกออฟ1ธ.ค. รวบ2ผู้ต้...,สปช.ทุ่มงบ 80 ล้าน เปิด 1.3 พันเวทีปฏิรูป รับฟ...,https://www.bangkokbiznews.com/news/detail/620023,7,"[ทุ่ม, ล้าน, เวที, จว.]",22,"[สปช., ลุย, ฟัง, ความเห็น, ปฏิรูป, คิก, ออฟ, ธ...",1381,"[สปช., ทุ่ม, งบ, ล้าน, พัน, เวที, ปฏิรูป, รับฟ..."
2,620025,2014-11-27 10:35:00,ธุรกิจ,NOW26ปักหลักสยามสแควร์ เปิดสตูดิโอใหม่พันตรม.-...,เปิดแผนปี 2558 ดิจิทัลทีวี NOW26 เตรียมเปิดสตู...,เพิ่มช่องทางเข้าถึงคนรุ่นใหม่ สร้างประสบการณ์ร...,https://www.bangkokbiznews.com/news/detail/620025,13,"[NOW, ปักหลัก, สยามสแควร์, สตูดิโอ, พัน, ตรม, ...",21,"[แผน, ปี, ดิจิทัล, ทีวี, NOW, เตรียม, สตูดิโอ,...",631,"[ช่องทาง, เข้าถึง, คนรุ่นใหม่, สร้าง, ประสบการ..."
3,620054,2014-11-27 11:08:00,การเงิน-การลงทุน,NBC - ซื้อ,เตรียมปรับเพิ่มค่าโฆษณาช่อง Nation TV ขึ้น 30-50%,ปรับเพิ่มมูลค่าเหมาะสมเป็น 6 บาท เราปรับเพิ่มม...,https://www.bangkokbiznews.com/news/detail/620054,3,"[NBC, ซื้อ]",12,"[เตรียม, ค่าโฆษณา, ช่อง, Nation, TV]",657,"[มูลค่า, เหมาะสม, บาท, มูลค่า, เหมาะสม, หุ้น, ..."
4,620065,2014-11-27 11:37:00,ธุรกิจ,'ซิคเว่'เสนอแผน'ดิจิทัลอีโคโนมี',"""ซิคเว่"" เล็งผนึก ""เอไอเอส ทรู"" พร้อมควง ""ทีโอ...","หวังสานฝันนโยบายรัฐสัมฤทธิผล รอนัดวันเข้าพบ ""ห...",https://www.bangkokbiznews.com/news/detail/620065,12,"[ซิ, คเว่', เสนอ, แผน, ดิจิทัล, อี, โค, โน]",28,"[ซิ, คเว่"", เล็ง, ผนึก, เอไอเอส, ทรู, ควง, ทีโ...",1329,"[หวัง, สาน, ฝัน, นโยบาย, รัฐ, สัมฤทธิผล, รอ, น..."


In [36]:
# Check shape
matichon.shape

(10433, 10)

In [37]:
# Check head
matichon.head()

,title,article,date,category,url,id,title_count,title_tokenize,article_count,article_tokenize
0,"แห่ชมฝนดาวตก ""เจมินิดส์""คึกคัก ""ดอยอินทนนท์"" ช...",วันที่ 15 ธันวาคม นายศรันย์ โปษยะจินดา รองผู้อ...,2015-12-15 02:36:22,region,https://www.matichon.co.th/region/news_622,622,20,"[แห่, ชม, ฝนดาวตก, เจ, ส์"", คึกคัก, ดอย, อินทน...",721,"[วันที่, ธันวาคม, นา, ยศ, รัน, ย์, โป, ษ, ยะ, ..."
1,สมาชิกรุมจวกกม.คลื่นความถี่! 'วิทยา' ซัดสปท.ไม...,ถก ร่างแก้ไข พ.ร.บ.องค์กรจัดสรรคลื่นความถี่ ทำ...,2016-01-18 23:48:28,politics,https://www.matichon.co.th/politics/news_5384,5384,19,"[สมาชิก, รุม, จวก, กม., คลื่นความถี่, วิทยา, ซ...",558,"[ถก, ร่าง, พ.ร.บ., องค์กร, จัดสรร, คลื่นความถี..."
2,โครงการ 3 ประสานความร่วมมือ กสทช. กสท และ CABSAT,โครงการ 3 ประสานความร่วมมือ กสทช. กสท และ CABS...,2016-01-20 09:08:59,publicize,https://www.matichon.co.th/publicize/news_6608,6608,8,"[โครงการ, ประสาน, ความร่วมมือ, กสทช., กสท, CAB...",425,"[โครงการ, ประสาน, ความร่วมมือ, กสทช., กสท, CAB..."
3,"เอไอเอส จัดเต็ม มหกรรม ""AIS 4G ADVANCED NEW WO...","เอไอเอสขอเชิญลูกค้าและประชาชน ร่วมงานมหกรรม ""A...",2016-01-24 09:08:54,publicize,https://www.matichon.co.th/publicize/news_11306,11306,14,"[เอไอเอส, เต็ม, มหกรรม, AIS, G, ADVANCED, NEW,...",475,"[เอไอเอส, เชิญ, ลูกค้า, ประชาชน, ร่วมงาน, มหกร..."
4,"CAT เปิดศูนย์นวัตกรรม ""IOT City Innovation Ce...",บมจ. กสท โทรคมนาคม (CAT) วางโครงข่ายระบบไอทีค...,2016-01-24 12:46:00,publicize,https://www.matichon.co.th/publicize/news_11463,11463,10,"[CAT, ศูนย์, นวัตกรรม, IOT, City, Innovation, ...",404,"[บมจ., กสท, โทรคมนาคม, CAT, วาง, โครงข่าย, ระบ..."


### 1.2 Bag-of-Words: Vectorize with DictVectorizer

Since computer does not understand words as human does, they need to be converted into numbers by vectorizing. However, for Thai language, DictVectorizer will be used as a vectorizer.

In [38]:
# Function to store n_word in dict
def featurize(token_list):
    
    '''
    Description:
    Store word that occurs in the corpus in dictionary form
    
    Parameters:
    List of tokens
    
    Returns:
    Dictionary - token as key and occurance (one) as value
    
    '''
    
    token_list=token_list
    features = {}
    for token in token_list:
        features[token]=1
    return features

In [39]:
# Featurize words in corpus
bkkbiz_token = bkkbiz['article_tokenize'].apply(featurize)
matichon_token = matichon['article_tokenize'].apply(featurize)

In [40]:
# Vectorize words in corpus with DictVectorizer and store them as a sparse matrix
vectorizer_1 = DictVectorizer(sparse=True)
vectorizer_2 = DictVectorizer(sparse=True)

bkkbiz_vec = vectorizer_1.fit_transform(bkkbiz_token)
matichon_vec = vectorizer_2.fit_transform(matichon_token)

In [41]:
# Check origital number of articles
bkkbiz.shape, matichon.shape

((16464, 13), (10433, 10))

In [42]:
# Compare the new vectorized value
bkkbiz_vec, matichon_vec

(<16464x63496 sparse matrix of type '<class 'numpy.float64'>'
 	with 2760975 stored elements in Compressed Sparse Row format>,
 <10433x59082 sparse matrix of type '<class 'numpy.float64'>'
 	with 2135687 stored elements in Compressed Sparse Row format>)

## 2. Model Comparison

### 2.1 K-Means

In [43]:
def kmeans(data_vec=None, n_clusters=10):
    '''
    Definition:
        - fit K-mean model
    Parameters:
        - vectorized data
        - number of clusters (number of topics)
    Returns:
        - fitted model
    '''
    
    kmeans = KMeans(n_clusters=n_clusters, 
                    init='k-means++', 
                    max_iter=100, 
                    n_init=1, 
                    random_state=42)

    model = kmeans.fit(data_vec)
    return model

In [44]:
def kmeans_topic(model=None, n_clusters=10, corpus=None, vectorizer=None):
    '''
    Definition:
        - list out the topic and its related words
    Parameters:
        - model
        - number of clusters (number of topics)
    Retuens:
        - topics with top 30 related words
    '''
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names_out()
    print(corpus)
    print('K-Means')
    print('*'*20)
    for i in range(n_clusters):
        print(f'Topic {i+1}')
        print('-'*10)
        words = []
        for i in order_centroids[i,:30]:
            words.append(terms[i])
        print(words)
        print(' ')

In [45]:
# Kmeans model result from Bkkbiz Corpus with 10 topics and 15 related words 
kmeans_bkkbiz = kmeans(data_vec=bkkbiz_vec, n_clusters=10)
kmeans_topic(model=kmeans_bkkbiz, n_clusters=10, corpus='BangkokBiz', vectorizer=vectorizer_1)

BangkokBiz
K-Means
********************
Topic 1
----------
['ดิจิทัล', 'ทักษะ', 'ความร่วมมือ', 'แพลตฟอร์ม', 'พันธุ์', 'บุคลากร', 'ขับเคลื่อน', 'บริษัท', '์', 'วัฒน', 'คน', 'การพัฒนา', 'นวัตกรรม', 'คลาวด์', 'ไมโครซอฟท์', 'สำหรับ', 'พัฒนา', 'เทคโนโลยี', 'สุ', 'ศักยภาพ', 'จำกัด', 'ดา', 'สร้าง', 'กรรมการผู้จัดการ', 'ไทย', 'หลัก', 'ธรรม', 'ความสามารถ', 'ระดับโลก', 'ประเทศ']
 
Topic 2
----------
['เนจ', 'BNK', 'บริษัท', 'สร้าง', 'นท', 'อิน', 'การพัฒนา', 'วงการ', 'จำกัด', 'iAM', 'เม้นท์', 'จับมือ', 'เด้', '์', 'เพ', 'น', 'เม', 'งวง', 'พัฒนา', 'ไทย', 'ไอดอล', 'กิจกรรม', 'อาร์', 'ดิจิทัล', 'ศักยภาพ', 'ทิสท์', 'ดิ', 'เป่ายิ้งฉุบ', 'โลกาภิวัตน์', 'น้อยลง']
 
Topic 3
----------
['ดิจิทัล', 'คาด', 'ปี', 'ล้าน', 'บาท', 'หุ้น', 'ราคา', 'ซื้อ', 'ไทย', 'ตลาด', 'เดือน', 'เพิ่มขึ้น', 'ธุรกิจ', 'ลดลง', 'บวก', 'เติบโต', 'การลงทุน', 'แนวโน้ม', 'กำไร', 'ต่อเนื่อง', 'ปัจจัย', 'จุด', 'รายได้', 'ขาย', 'ลด', 'ระดับ', 'หนุน', 'ปรับตัว', 'บริษัท', 'มูลค่า']
 
Topic 4
----------
['ดิจิทัล', 'ปี', 'ไทย', 'ธุรกิจ', '

In [90]:
# Save model
filename = 'model_kmean_bkkbiz.joblib'
joblib.dump(kmeans_bkkbiz, filename)

['model_kmean_bkkbiz.joblib']

In [46]:
# Kmeans model result from Matichon Corpus with 10 topics and 15 related words 
kmeans_matichon = kmeans(data_vec=matichon_vec, n_clusters=10)
kmeans_topic(model=kmeans_matichon, n_clusters=10, corpus='Matichon', vectorizer=vectorizer_2)

Matichon
K-Means
********************
Topic 1
----------
['ดิจิทัล', 'ไทย', 'สำหรับ', 'ปี', 'บริษัท', 'ประเทศ', 'เทคโนโลยี', 'สร้าง', 'จำกัด', 'คน', 'ดี', 'ยุค', 'ทำ', 'ธุรกิจ', 'ที่จะ', 'งาน', 'ระบบ', 'พัฒนา', 'โลก', 'บริการ', 'กล่าวว่า', 'ข้อมูล', 'วันที่', 'เรื่อง', 'รูปแบบ', 'ลูกค้า', 'ออนไลน์', 'บาท', '์', 'มากขึ้น']
 
Topic 2
----------
['คน', 'เรื่อง', 'ไทย', 'วันที่', 'ประเทศ', 'ดิจิทัล', 'ประชาชน', 'รัฐบาล', 'ปี', 'ทำ', 'อ.', 'พล', 'ดี', 'ที่จะ', 'จันทร์', 'เวลา', 'สังคม', 'ประยุทธ์', 'รัฐมนตรี', 'ว่าการ', 'นายกรัฐมนตรี', 'ประธาน', 'สร้าง', 'โอชา', 'สำหรับ', 'กฎหมาย', 'ปัญหา', 'กล่าวว่า', 'ระบบ', 'เข้ามา']
 
Topic 3
----------
['มหาวิทยาลัย', 'การทำสัญญา', 'ที่อยู่', 'ติดต่อ', 'ยอดขาย', 'อนัญ', 'ไทยใหญ่', 'ไทยแลนด์', 'ศาสตราจารย์', 'ไทย', 'อนาคต', 'ตารางเมตร', 'บางส่วน', 'บางคน', 'ยังอยู่', 'Generation', 'อบ', 'TCS', 'ฟัง', 'ยกระดับ', 'ตี', 'ป๋วย', 'การทำงาน', 'ทีมงาน', 'ศตวรรษ', 'สุ่ม', 'ที่จะ', 'การตัดสินใจ', 'เสี่ยง', 'เข้ามา']
 
Topic 4
----------
['โทรคมนาคม', 'กิจการ', '

In [89]:
# Save model
filename = 'model_kmean_matichon.joblib'
joblib.dump(kmeans_matichon, filename)

['model_kmean_matichon.joblib']

**Key Findings - KMean**
- KMeans match a text to a topic. One text belongs only to one topic, whereas a topic can have a multiple matching text.
- It seems that K-mean is successful at extracting topic and relevant keywords, however, labeling the topics is still not easy to perform, as there are many possibilies of choice for a K-mean topic. 

### 2.2 LDA

In [47]:
# Function for modeling with LDA
def lda_model(data=None, num_topics=None): #, topicid=None
    
    '''
    Description:
    Fit documents to LDA model
    
    Parameters:
    - data (list)
    - number of topics
    
    Returns:
    - Perplexity rate
    - Topic and its top 15 associated words
    
    '''
    
    dictionary = Dictionary(data)
    corpus = [dictionary.doc2bow(txt) for txt in data]
    
    model = LdaModel(corpus=corpus, num_topics=num_topics)
    perplexity = model.log_perplexity(corpus)
    print(f'LDA Model with {num_topics} topics')
    print(f'Perplexity: {perplexity.round(2)}')
    print('-'*60)
    for i in range(num_topics):
        top_n = [dictionary[index] for index, prob in model.get_topic_terms(topicid=i, topn=30)]
        print(f'Topic {i+1}')
        print(top_n)
        print('-'*60)
    return model, corpus, dictionary

In [48]:
# Visualization
#def lda_vis(data=None, num_topics=20):
def lda_vis(model=None, corpus=None, dictionary=None):    
    '''
    Description:
    Visualize LDA model
    
    Parameters:
    Fitted model, corpus and dictionary
    
    Returns:
    LDA visualisation
    
    '''
    
    #dictionary = Dictionary(data)
    #corpus = [dictionary.doc2bow(txt) for txt in data]
    #model = LdaModel(corpus=corpus, num_topics=num_topics)
    model=model
    pyLDAvis.enable_notebook()
    viz = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    return viz

In [49]:
# LDA model with BangkokBiz corpus
bkkbiz_lda_model, bkkbiz_lda_corpus, bkkbiz_lda_dictionary  = lda_model(data=bkkbiz['article_tokenize'], num_topics=10)

LDA Model with 10 topics
Perplexity: -8.22
------------------------------------------------------------
Topic 1
['ธุรกิจ', 'บริษัท', 'ดิจิทัล', 'ลูกค้า', 'ไทย', 'ปี', 'บริการ', 'เทคโนโลยี', 'จำกัด', 'สร้าง', 'องค์กร', 'เติบโต', 'ล้าน', 'ธนาคาร', 'ระบบ', 'ตลาด', 'ประเทศ', 'สำหรับ', 'ให้บริการ', 'ทำ', 'มากขึ้น', '์', 'บริหาร', 'เข้ามา', 'อี', 'พัฒนา', 'ข้อมูล', 'สินค้า', 'โควิด', 'กล่าวว่า']
------------------------------------------------------------
Topic 2
['ล้าน', 'ปี', 'บาท', 'ออนไลน์', 'สินค้า', 'ดิจิทัล', 'ธุรกิจ', 'เติบโต', 'สื่อ', 'ไทย', 'ผู้บริโภค', 'บริษัท', 'แบรนด์', 'แพลตฟอร์ม', 'ตลาด', 'ทำ', 'สร้าง', 'ช่อง', 'ทีวี', 'รายได้', 'โฆษณา', 'มูลค่า', 'เพิ่มขึ้น', 'ซื้อ', 'ประเทศ', 'ช่องทาง', 'เดือน', 'คน', 'มากขึ้น', 'ลูกค้า']
------------------------------------------------------------
Topic 3
['บาท', 'ล้าน', 'ปี', 'ราคา', 'หุ้น', 'ดิจิทัล', 'สลาก', 'สินทรัพย์', 'เงิน', 'คาด', 'ไทย', 'ตลาด', 'เดือน', 'ลดลง', 'เพิ่มขึ้น', 'สกุล', 'ซื้อ', 'ดอลลาร์', 'ระดับ', 'การลงทุน', 'บิท', 'รา

In [50]:
# Visualization
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

lda_vis(model=bkkbiz_lda_model, corpus=bkkbiz_lda_corpus, dictionary=bkkbiz_lda_dictionary)

/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.132948 -0.045159       1        1  20.268118
5     -0.068641 -0.076993       2        1  13.867159
1     -0.117474 -0.071082       3        1  12.237041
6      0.014807 -0.001379       4        1  11.865302
2     -0.119355  0.104985       5        1  10.979491
4      0.104399 -0.157270       6        1   9.440826
8      0.090880  0.136226       7        1   7.753998
3     -0.075705  0.107730       8        1   5.865184
9      0.214455  0.038780       9        1   4.532430
7      0.089582 -0.035838      10        1   3.190451, topic_info=           Term          Freq         Total Category  logprob  loglift
77          บาท  31119.000000  31119.000000  Default  30.0000  30.0000
181    เศรษฐกิจ  18688.000000  18688.000000  Default  29.0000  29.0000
120        ล้าน  34676.000000  34676.000000  Default  28.0000  28.0000
10     การลงทุน  13946.000000  13946.000000  Default  27.0000  27.0000
156  อุตสาหกรรม  12654.000000  12654.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
47      ดิจิทัล    636.951836  58521.063136  Topic10  -5.6341  -1.0754
580         ไทย    617.296281  51700.281923  Topic10  -5.6654  -0.9829
34        จำนวน    519.917986  12697.438408  Topic10  -5.8371   0.2495
81           ปี    529.834188  48172.526597  Topic10  -5.8182  -1.0650
120        ล้าน    484.791851  34676.050361  Topic10  -5.9070  -0.8251

[931 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1989      1  0.007566  ADVANC
1989      2  0.001513  ADVANC
1989      3  0.055992  ADVANC
1989      4  0.001513  ADVANC
1989      5  0.915546  ADVANC
...     ...       ...     ...
999       8  0.031912       ์
999       9  0.128033       ์
999      10  0.025545       ์
6669      1  0.947359    ์ตี้
6669      2  0.018947    ์ตี้

[4965 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 2, 7, 3, 5, 9, 4, 10, 8])

In [88]:
# Save model
filename = 'model_lda_bkkbiz.joblib'
joblib.dump(bkkbiz_lda_model, filename)

['model_lda_bkkbiz.joblib']

In [51]:
# LDA model with matichon dataset
matichon_lda_model, matichon_lda_corpus, matichon_lda_dictionary = lda_model(data=matichon['article_tokenize'], num_topics=10)

LDA Model with 10 topics
Perplexity: -8.33
------------------------------------------------------------
Topic 1
['ข้อมูล', 'วันที่', 'ประชาชน', 'เรื่อง', 'คน', 'กฎหมาย', 'สิทธิ', 'คดี', 'จำนวน', 'ตรวจสอบ', 'ดำเนินการ', 'มาตรา', 'กรณี', 'ศาล', 'แจ้ง', 'สถานการณ์', 'ดี', 'คณะกรรมการ', 'ดิจิทัล', 'หน่วยงาน', 'สำนักงาน', 'พ.ร.บ.', 'อี', 'พ.ศ.', 'เจ้าหน้าที่', 'คุ้มครอง', 'ปี', 'พิจารณา', 'บุคคล', 'ยาเสพติด']
------------------------------------------------------------
Topic 2
['ธุรกิจ', 'ออนไลน์', 'ดิจิทัล', 'สร้าง', 'ลูกค้า', 'เทคโนโลยี', 'ปี', 'ไทย', 'แพลตฟอร์ม', 'องค์กร', 'แบรนด์', 'ข้อมูล', 'สำหรับ', 'บริการ', 'บริษัท', 'ผู้บริโภค', 'การทำงาน', 'สื่อ', 'ทำ', 'ประสบการณ์', 'มากขึ้น', 'โลก', 'สินค้า', 'รูปแบบ', 'เติบโต', 'ที่จะ', 'เข้าถึง', 'คน', 'LINE', 'ดี']
------------------------------------------------------------
Topic 3
['ระบบ', 'ข้อมูล', 'เทคโนโลยี', 'ไทย', 'ความปลอดภัย', 'ดิจิทัล', 'องค์กร', 'สำหรับ', 'โควิด', 'ประเทศ', 'อุปกรณ์', 'การทำงาน', 'บริการ', 'พื้นที่', 'อี', 'ไฟฟ้า',

In [52]:
# Visualization
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

lda_vis(model=matichon_lda_model, corpus=matichon_lda_corpus, dictionary=matichon_lda_dictionary)

/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.004538  0.142702       1        1  13.050182
4      0.013038  0.031607       2        1  12.519858
5      0.025104 -0.008801       3        1  11.792228
6     -0.161586 -0.026383       4        1  11.476743
1      0.108859 -0.027516       5        1  10.453397
9     -0.051392  0.113435       6        1  10.403656
2      0.047113 -0.007328       7        1   8.565041
0     -0.184690 -0.090278       8        1   8.063018
8      0.098054 -0.089833       9        1   7.664149
7      0.100962 -0.037605      10        1   6.011728, topic_info=           Term          Freq         Total Category  logprob  loglift
437      ลูกค้า  10973.000000  10973.000000  Default  30.0000  30.0000
1667   การศึกษา   6749.000000   6749.000000  Default  29.0000  29.0000
2346        ครู   4794.000000   4794.000000  Default  28.0000  28.0000
556         บาท  12652.000000  12652.000000  Default  27.0000  27.0000
413      ธุรกิจ  21079.000000  21079.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
1372     รายได้    665.715159   4681.577119  Topic10  -5.8263   0.8609
154       สร้าง    858.623504  19350.312821  Topic10  -5.5718  -0.3037
395       จำกัด    705.734503   8601.497580  Topic10  -5.7679   0.3110
76    ที่ผ่านมา    668.085127   6826.934166  Topic10  -5.8228   0.4872
55      ดิจิทัล    639.158956  25554.784565  Topic10  -5.8670  -0.8770

[939 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
27426      1  0.931057  ACMECS
27426      6  0.048158  ACMECS
476        1  0.032037     AIS
476        2  0.019863     AIS
476        3  0.016659     AIS
...      ...       ...     ...
14990      6  0.011419      ⦁…
14990      7  0.007613      ⦁…
14990      8  0.022838      ⦁…
14990      9  0.015226      ⦁…
14990     10  0.003806      ⦁…

[4727 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 6, 7, 2, 10, 3, 1, 9, 8])

In [87]:
# Save model
filename = 'model_lda_matichon.joblib'
joblib.dump(matichon_lda_model, filename)

['model_lda_matichon.joblib']

**Key Findings - LDA**
- LDA is a popular algorithm for topic-modeling. It extracts topics from documents, where one documents might show multiple topics. This is the point where LDA differentiate itself from KMeans. 
- However, one of the biggest challenges of LDA is that users have to define the number of topic by him/herself. 
- Considering the number of topics might be unknown, we might missed some insight from tuning the false number of topics.
- The visualization with pyLDAvis helps a lot to identify the ideal number of topics, where the more overlaps, the more we should consider reducing the number of topics. 
- However, we can see form both visualization that even with 10 topics, there are sill overlaps and the topics are at the same time not successful in filtering the articles related to Digital Competitiveness Ranking.

### 2.3 NMF

In [55]:
# Create function for modeling with NMF
def nmf(data=None, n_components=10, vectorizer=None):
    
    '''
    Description:
    Fit documents to NMF model
    
    Parameters:
    - Data (vectorized)
    - Number of components (number of topics)
    - Vectorizer
    
    Returns:
    Fitted model
    
    '''
    
    model = NMF(n_components=n_components)
    model.fit(data)
    features = model.transform(data)
    components = pd.DataFrame(model.components_,
                                columns=vectorizer.get_feature_names())
    # Topic & Words
    
    for i in range(n_components):
        words = []
        topic = components.iloc[i]
        print(f'Top 15 words for topic {i+1} are:')
        for n, word in enumerate(topic.nlargest(30)):
            words.append(topic.nlargest(30).keys()[n])
        print(words)
        print('\n')
    return model

In [57]:
bkkbiz_nmf_model = nmf(data=bkkbiz_vec, n_components=10, vectorizer=vectorizer_1)

Top 15 words for topic 1 are:
['คน', 'เรื่อง', 'ตัวเอง', 'ทำ', 'ดู', 'รู้', 'ตัว', 'ดี', 'เวลา', 'ทำงาน', 'หา', 'เหมือน', 'บอ', 'เกิดขึ้น', 'ต้องการ', 'งาน', 'โลก', 'เข้ามา', 'เลือก', 'เป็นเรื่อง', 'ที่จะ', 'บ้าน', 'ชีวิต', 'มากขึ้น', 'กลายเป็น', 'ออกมา', 'วิธี', 'สร้าง', 'ไม่ต้อง', 'ยุค']


Top 15 words for topic 2 are:
['วันที่', 'ประชาชน', 'คณะกรรมการ', 'เรื่อง', 'ดำเนินการ', 'แห่งชาติ', 'รัฐบาล', 'รัฐมนตรี', 'อ.', 'ดี', 'กล่าวว่า', 'พล', 'พิจารณา', 'ว่าการ', 'กรณี', 'นายกรัฐมนตรี', 'ดิจิทัล', 'กฎหมาย', 'ประธาน', 'สำนักงาน', 'หน่วยงาน', 'ไทย', 'เสนอ', 'กระทรวง', 'สังคม', 'จันทร์', 'อี', 'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม', 'ต่อไป', 'ตรวจสอบ']


Top 15 words for topic 3 are:
['คาด', 'ราคา', 'บาท', 'หุ้น', 'ลดลง', 'เพิ่มขึ้น', 'ล้าน', 'ตลาด', 'ซื้อ', 'เดือน', 'ปี', 'บวก', 'ปัจจัย', 'การลงทุน', 'แนวโน้ม', 'ปรับตัว', 'ขาย', 'กำไร', 'มูลค่า', 'ระดับ', 'ลด', 'เศรษฐกิจ', 'เติบโต', 'จุด', 'รายได้', 'นักลงทุน', 'ไตรมาส', 'อัตรา', 'หนุน', 'หมื่น']


Top 15 words for topic 4 are:
['พายุ', 'ส

In [86]:
# Save model
filename = 'model_nmf_bkkbiz.joblib'
joblib.dump(bkkbiz_nmf_model, filename)

['model_nmf_bkkbiz.joblib']

In [58]:
matichon_nmf_model = nmf(data=matichon_vec, n_components=10, vectorizer=vectorizer_2)

Top 15 words for topic 1 are:
['เทคโนโลยี', 'องค์กร', 'ธุรกิจ', 'บริษัท', 'พัฒนา', 'นวัตกรรม', 'บริการ', 'สร้าง', 'การพัฒนา', 'ระบบ', 'ข้อมูล', 'ที่จะ', 'การทำงาน', 'มีประสิทธิภาพ', 'ก้าว', 'ดิจิทัล', 'ยุค', 'รูปแบบ', 'ให้บริการ', 'ประเทศ', 'การนำ', 'ขับเคลื่อน', 'ลูกค้า', 'อุตสาหกรรม', 'บริหาร', 'จำกัด', 'เครือข่าย', 'ตอบ', 'ในอนาคต', 'ศักยภาพ']


Top 15 words for topic 2 are:
['วันที่', 'รัฐมนตรี', 'ว่าการ', 'อ.', 'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม', 'นายกรัฐมนตรี', 'พล', 'ประยุทธ์', 'จันทร์', 'ประชาชน', 'โอชา', 'กระทรวง', 'ดี', 'เรื่อง', 'อี', 'รัฐบาล', 'กล่าวว่า', 'น.', 'เวลา', 'การประชุม', 'กฎหมาย', 'พงษ์', 'ดำเนินการ', 'ประเทศ', 'ปุ', 'ประธาน', 'กันต์', 'ณณ', 'แห่งชาติ', 'พุทธิ']


Top 15 words for topic 3 are:
['คน', 'ทำ', 'เรื่อง', 'เกิดขึ้น', 'รู้', 'ตัวเอง', 'เป็นเรื่อง', 'ดู', 'ออกมา', 'โลก', 'ที่จะ', 'เข้ามา', 'เหมือน', 'ตัว', 'บอ', 'ปัญหา', 'เวลา', 'หา', 'มากขึ้น', 'กลายเป็น', 'มาจาก', 'ผม', 'เข้าไป', 'เดิม', 'ต้องการ', 'ดี', 'ยุค', 'ดังนั้น', 'ทำงาน', 'เลือก']


Top 15 

In [85]:
# Save model
filename = 'model_nmf_matichon.joblib'
joblib.dump(matichon_nmf_model, filename)

['model_nmf_matichon.joblib']

**Key Findings**
- NMF model calculates topic x terms x documents together. 
- However, it shows the same problem as LDA that it is not easy to lable a topic with a title.

### 2.4 Top2Vec

In [59]:
# Create function that tokenize, remove stopwords and perform RegularExpression
def tokenize(txt):
  
    ''' 
    Description:
        A function that returns tokenized words with stop words and other regular expressions 
        that do not match English and Thai alphabets removed

    Parameters:
        txt = text string

    Return:
        word tokens as list
    '''

    stopwords = thai_stopwords()
    tokens = word_tokenize(txt, keep_whitespace = False)
    tokens = [token for token in tokens if token not in stopwords]
    tokens = [token for token in tokens if re.match(r'[A-Za-zก-\uDe5c]+', token)]
    return tokens

In [60]:
# Create function for modeling with Top2Vec
def top2vec(documents=None, speed='fast-learn', ngram_vocab=None):
    
    '''
    Description:
        - fit Top2Vec model
    Parameters:
        - documents(in list format)
        - number of topics
        - learning speed: fast-learn, learn or deep-learn
    Return:
        - model
        - topic numbers
        - topic words
    '''
    
    model = Top2Vec(documents=documents,
                      speed=speed,
                      tokenizer=tokenize)
    topic_words, word_scores, topic_nums = model.get_topics()
    print(f'Topic: {topic_nums+1}')
    print(f'Words: {topic_words}')
    return model, topic_nums, topic_words

In [61]:
def top2vec_doc(model=None, topic_num=None):
    documents, document_scores, document_ids = \
    model.search_documents_by_topic(topic_num=topic_num, num_docs=1) # Reduce to 1 to save space on GitHub
    for doc, score, doc_id in zip(documents, document_scores, document_ids):
        print(f'Document: {doc_id}, Score: {score}')
        print('-'*20)
        print(doc)
        print('-'*20)
        print()

In [62]:
# Prepare input data as list
bkkbiz_list = bkkbiz['article'].tolist()
matichon_list = matichon['article'].tolist()

**Reference for variable names**

|number|speed|
|---|---|
|bX|Bkkbiz|
|mX|Matichon|
|X1|fast-learn|
|X2|learn|
|X3|deep-learn|


e.g. 
- model_b1 = Top2Vec model with fast-learn speed fitted on BangkokBiz corpus
- model_m3 = Top2Vec model with deep-learn speed fitted on Matichon corpus

**2.4.1 Fast-learn model**

In [14]:
# Bkkbiz Fast-learn model
model_b1 = Top2Vec(documents=bkkbiz_list,speed='fast-learn',tokenizer=tokenize)
b1_topic_words,b1_word_scores, b1_topic_nums = model_b1.get_topics()

2022-11-21 01:52:28,468 - top2vec - INFO - Pre-processing documents for training
2022-11-21 01:54:37,132 - top2vec - INFO - Creating joint document/word embedding
2022-11-21 02:04:11,588 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-21 02:04:28,862 - top2vec - INFO - Finding dense areas of documents
2022-11-21 02:04:31,784 - top2vec - INFO - Finding topics


In [65]:
# Calling topic and related words from BangkokBiz corpus from fast-learn model
b1_topic_words[0] # Show only the result from index [0] (= Topic 1) to save space on GitHub 

array(['ธนาคาร', 'สินเชื่อ', 'งกิ้ง', 'ธนาคารพาณิชย์', 'แบงก์',
       'ทางการเงิน', 'แบงก์กิ้ง', 'กสิกร', 'Banking', 'SCB',
       'ค่าธรรมเนียม', 'NPL', 'NIM', 'กรุงศรี', 'สถาบันการเงิน',
       'เงินฝาก', 'Lending', 'ลูกค้า', 'ธนาคารแห่งประเทศไทย', 'ดอกเบี้ย',
       'ธุรกรรม', 'แบ', 'กรุงศรีอยุธยา', 'เอ็นพีแอล', 'หนี้สูญ', 'EASY',
       'Financial', 'หนี้', 'Krungsri', 'ttb', 'การเงิน', 'KBANK',
       'อัตราดอกเบี้ย', 'SME', 'เงินสด', 'สินทรัพย์', 'หนี้เสีย',
       'ลูกหนี้', 'พาณิชย์', 'ชำระเงิน', 'บัญชีเงินฝาก', 'สิ้นปี',
       'เปิดบัญชี', 'เงินได้', 'Bank', 'ธปท.', 'บัตรเครดิต', 'กรุงไทย',
       'เชื่อได้', 'ผยง'], dtype='<U48')

In [66]:
# Call topic related documents
top2vec_doc(model=model_b1, topic_num=1) # Show only example document from topic 1 to save space on Github

Document: 16138, Score: 0.8035717010498047
--------------------
นายนพดล ปัญญาธิปัตย์ กรรมการผู้จัดการประจำประเทศไทย เดลล์ เทคโนโลยีส์ กล่าวว่า โลกธุรกิจปัจจุบันมีความซับซ้อนเพิ่มมากขึ้น ต้องเผชิญกับปัจจัยโรคระบาด จำเป็นต้องมองหาสิ่งใหม่ๆ ที่จะทำให้เกิดโอกาสการเติบโต


ดังนั้นระบบไอทีต้องมีการยกระดับสู่รูปแบบอัตโนมัติ ให้ความสำคัญกับไซเบอร์ซิเคียวริตี้ ขณะเดียวกันมีความสามารถที่จะบริหารจัดการมัลติคลาวด์

เขากล่าวว่า องค์กรเริ่มหันมาให้ความสำคัญกับการบริหารจัดการข้อมูลมากขึ้น โดยที่เริ่มดำเนินการแล้วเช่น ธุรกิจด้านการเงินการธนาคาร โทรคมนาคม และต่อไปที่จะแพร่หลายมากขึ้นคาดว่าจะเป็นกลุ่มอีคอมเมิร์ซและดิลิเวอรีเพื่อการเข้าถึงความต้องการของลูกค้าได้ดีมากขึ้น รวมถึงรองรับบริหารจัดการฐานลูกค้าที่มีมากขึ้น

"เราได้เห็นองค์กรพลิกโฉมระบบไอทีเพื่อรองรับการเปลี่ยนแปลงหรือดิสรัปชันต่างๆ ดังที่จะเห็นได้จากช่วงสองสามปีที่ผ่านมาที่องค์กรได้เริ่มวางแผนและลงมือทำดิจิทัลทรานส์ฟอร์เมชันอย่างเป็นรูปธรรม"
--------------------



In [91]:
# Save model
filename = 'model_ttvb1.joblib'
joblib.dump(model_b1, filename)

['model_ttvb1.joblib']

In [21]:
# Matichon Fast-learn model
model_m1 = Top2Vec(documents=matichon_list,speed='fast-learn',tokenizer=tokenize)
m1_topic_words,m1_word_scores, m1_topic_nums = model_m1.get_topics()

2022-11-21 02:08:51,492 - top2vec - INFO - Pre-processing documents for training
2022-11-21 02:10:07,019 - top2vec - INFO - Creating joint document/word embedding
2022-11-21 02:15:44,531 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-21 02:15:48,280 - top2vec - INFO - Finding dense areas of documents
2022-11-21 02:15:48,548 - top2vec - INFO - Finding topics


In [67]:
# Calling topic and related words from BangkokBiz corpus from fast-learn model
m1_topic_words[0] # Show only the result from index [0] (topic 1) to save space on GitHub 

array(['ดำเนินคดี', 'กระทำความผิด', 'ความผิด', 'เข้าข่าย', 'แจ้งความ',
       'คำสั่งศาล', 'ปราบปราม', 'พนักงานสอบสวน', 'ผิดกฎหมาย', 'จับกุม',
       'โพสต์', 'การกระทำ', 'บังคับการ', 'พ.ต.อ.', 'ท.)', 'การจับกุม',
       'พยานหลักฐาน', 'ปอ', 'เจ้าหน้าที่ตำรวจ', 'ปลอม', 'ตามกฎหมาย',
       'เท็จ', 'สำนักงานตำรวจแห่งชาติ', 'บิดเบือน', 'มีความผิด',
       'ผู้ต้องหา', 'ผู้เสียหาย', 'ท.', 'ผู้กระทำผิด', 'ปิดกั้น',
       'การปิดกั้น', 'ศาล', 'ประมวลกฎหมายอาญา', 'หลอกลวง', 'เพจ',
       'หมายศาล', 'ข้อความ', 'การสืบสวน', 'เอาผิด', 'อาชญากรรม', 'ข้อหา',
       'สืบสวน', 'หมายจับ', 'ตำรวจ', 'ระงับ', 'พล.ต.ต.', 'บก', 'เฟซบุ๊ก',
       'มีโทษ', 'พุทธิ'], dtype='<U41')

In [68]:
# Call topic related documents
top2vec_doc(model=model_m1, topic_num=1) # Show only example document from topic 1 to save space on Github

Document: 3172, Score: 0.7673107385635376
--------------------
นายรพี สุจริตกุล เลขาธิการสำนักงานคณะกรรมการกำกับหลักทรัพย์และตลาดหลักทรัพย์(ก.ล.ต.) กล่าวว่า สถานการณ์เงินทุนไหลออกในตลาดหุ้นช่วงนี้มองว่าเป็นผลกระทบระยะสั้น แต่ท้ายที่สุดแล้วก็จะต้องดูพื้นฐานของบริษัทหรือหุ้นที่ลงทุนมากกว่า แนวโน้มการดำเนินงานเป็นอย่างไรและบริษัทมีทิศทางการดำเนินงานอย่างไร เพราะหากพื้นฐานดีย่อมมีเม็ดเงินเข้ามาลงทุน ซึ่งเรื่องที่สำคัญคือการปรับตัวรองรับกับโลกดิจิทัลที่จะเข้ามา เพราะธุรกิจเดิมจะถูกกระทบและทำให้เกิดธุรกิจใหม่ หากบริษัทไม่ปรับตัวก็จะไม่สามารถอยู่รอดได้
" ขณะนี้ไม่เห็นแนวโน้มว่าจะเกิดความเสียหายหรือเกิดวิกฤติเศรษฐกิจซ้ำรอยอีก เพราะครั้งนี้นเกิดจากสถาบันการเงินที่มีการปล่อยกู้และมีหนี้เสียจำนวนมาก แต่ปัจจุบันสถาบันการเงินแข็งแกร่ง เงินกองทุนสูง และหนี้เสียอยู่ในระดับต่ำ อย่างไรก็ดีมีปัจจัยที่กังวล แต่ไม่ใช่ธุรกิจการเงินเท่านั้น แต่คือทุกธุรกิจที่จะต้องมีการปรับตัวรับเทคโนโลยีที่เปลี่ยนแปลงไป เพราะหากไม่ปรับตัวก็จะมีธุรกิจใหม่เข้ามาแทนที่ล้มหายตายจากไป" นายรพี กล่าว
--------------------



In [92]:
# Save model
filename = 'model_ttvm1.joblib'
joblib.dump(model_m1, filename)

['model_ttvm1.joblib']

**2.4.2 Model with speed=learn**

In [24]:
# Bkkbiz 'learn' model
model_b2 = Top2Vec(documents=bkkbiz_list,speed='learn',tokenizer=tokenize)
b2_topic_words,b2_word_scores, b2_topic_nums = model_b2.get_topics()

2022-11-21 02:15:48,873 - top2vec - INFO - Pre-processing documents for training
2022-11-21 02:17:58,941 - top2vec - INFO - Creating joint document/word embedding
2022-11-21 02:33:19,781 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-21 02:33:36,815 - top2vec - INFO - Finding dense areas of documents
2022-11-21 02:33:41,013 - top2vec - INFO - Finding topics


In [93]:
# Calling topic and related words from BangkokBiz corpus from 'learn' model
b2_topic_words[0] # Show only the result from index [0] (topic 1) to save space on GitHub 

array(['องค์กร', 'ไอที', 'คอมพิวติ้ง', 'ไอ', 'ทรานส์', 'ซับซ้อน',
       'คลาวด์', 'ความยืดหยุ่น', '์ท', 'ฟอ', 'ออราเคิล', 'เอ', 'ค็อก',
       'การทำงาน', 'ดจ์', 'ยืดหยุ่น', 'ไฮบริด', 'ปัญญาประดิษฐ์', 'ทิฟ',
       'ไอบีเอ็ม', 'เลิร์น', 'โซลูชั่น', 'การประมวลผล', 'ธิงส์(',
       'เทคโนโลยี', 'สภาพแวดล้อม', 'แมชชีน', 'ร์', 'ธิงส์',
       'การเปลี่ยนแปลง', 'โอที', 'ปรับเปลี่ยน', 'ควอนตัม', 'ไล', 'เคียว',
       'สฟอร์', 'จำเป็นต้อง', 'ปฐ', 'คล่องตัว', 'นิง', 'แปซิฟิก',
       'ความคาดหวัง', 'เข้ากับ', 'กุญแจ', 'ผลลัพธ์', 'พนักงาน', 'ชั่น',
       'ความสามารถ', 'เทรนด์', 'ตื่นตัว'], dtype='<U48')

In [94]:
# Call topic related documents
top2vec_doc(model=model_b2, topic_num=1) # Show only example document from topic 1 to save space on Github

Document: 14502, Score: 0.7961863875389099
--------------------
การปรับตัวลงของบิตคอยน์ในช่วงเช้าวันนี้ เป็นการร่วงลงต่อเนื่องหลังจากเมื่อวันพฤหัสบดี(9ธ.ค.)บิตคอยน์ร่วงลงทะลุระดับ 50,000 ดอลลาร์ ท่ามกลางความกังวลเกี่ยวกับการที่ทางการสหรัฐจะเข้าควบคุมการซื้อขายสกุลเงินดิจิทัล

เมื่อเวลา 19.25 น.ตามเวลาไทย บิตคอยน์ดิ่งลง 2.73% สู่ระดับ 49,383.89 ดอลลาร์ในการซื้อขายบนแพลตฟอร์ม Coinbase

นักลงทุนกังวลว่า สหรัฐจะคุมเข้มการทำธุรกรรมสกุลเงินคริปโทฯ หลังจากที่สภาคองเกรสเรียกผู้บริหารของแพลตฟอร์มซื้อขายสกุลเงินดิจิทัลรายใหญ่ รวมทั้งผู้ที่เกี่ยวข้องกับธุรกิจขุดเหมืองบิตคอยน์เข้าให้การต่อคณะกรรมาธิการบริการการเงินประจำสภาผู้แทนราษฎรเมื่อวานนี้

ขณะนี้ทางการสหรัฐกำลังพิจารณาเกี่ยวกับการควบคุมธุรกรรมสกุลเงินดิจิทัล โดยอาจให้อยู่ภายใต้การกำกับดูแลของสำนักงานคณะกรรมการกำกับหลักทรัพย์และตลาดหลักทรัพย์ของสหรัฐ (กลต.) หรืออาจมีการตั้งหน่วยงานใหม่ขึ้นมากำหนดกฎระเบียบควบคุมธุรกิจดังกล่าว

นอกจากนี้ บิตคอยน์ยังถูกกดดันจากความกังวลที่ว่าสหรัฐอาจเรียกเก็บภาษีสกุลเงินคริปโทฯ

ทั้งนี้ นักลงทุนวิตกว่าสหรัฐจะทำก

In [95]:
# Save model
filename = 'model_ttvb2.joblib'
joblib.dump(model_b2, filename)

['model_ttvb2.joblib']

In [27]:
# Matichon 'learn' model
model_m2 = Top2Vec(documents=matichon_list,speed='learn',tokenizer=tokenize)
m2_topic_words,m2_word_scores, m2_topic_nums = model_m2.get_topics()

2022-11-21 02:33:42,478 - top2vec - INFO - Pre-processing documents for training
2022-11-21 02:35:37,268 - top2vec - INFO - Creating joint document/word embedding
2022-11-21 02:46:23,487 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-21 02:46:28,033 - top2vec - INFO - Finding dense areas of documents
2022-11-21 02:46:28,335 - top2vec - INFO - Finding topics


In [96]:
# Calling topic and related words from BangkokBiz corpus from 'learn' model
m2_topic_words[0] # Show only the result from index [0] (topic 1) to save space on GitHub 

array(['ความผิด', 'ดำเนินคดี', 'คำสั่งศาล', 'กระทำความผิด', 'บิดเบือน',
       'การกระทำ', 'มีความผิด', 'การปิดกั้น', 'เข้าข่าย',
       'ประมวลกฎหมายอาญา', 'ปิดกั้น', 'โพสต์', 'มีโทษ', 'ผิดกฎหมาย',
       'ปลุกปั่น', 'เอาผิด', 'เพจ', 'ระงับ', 'หมิ่น', 'เท็จ',
       'ผู้กระทำผิด', 'ฝ่าฝืน', 'ลบ', 'แจ้งความ', 'หมายศาล', 'ตามกฎหมาย',
       'อาญา', 'ตื่นตระหนก', 'ศาล', 'ปลอม', 'พยานหลักฐาน', 'ท.',
       'ข้อความ', 'จำคุก', 'ข้อกฎหมาย', 'ความสงบเรียบร้อย', 'เจตนา',
       'หมิ่นประมาท', 'คอมพิวเตอร์', 'ทำผิด', 'ทวิตเตอร์', 'ทางกฎหมาย',
       'เจ้าหน้าที่ตำรวจ', 'กระทำการ', 'ความร้ายแรง', 'การจับกุม',
       'เบาะแส', 'มาตรา', 'เฟค', 'ฟ้อง'], dtype='<U48')

In [97]:
# Call topic related documents
top2vec_doc(model=model_b2, topic_num=1) # Show only example document from topic 1 to save space on Githubtop2vec_doc(model=model_m2, topic_num=1) 

Document: 14502, Score: 0.7961863875389099
--------------------
การปรับตัวลงของบิตคอยน์ในช่วงเช้าวันนี้ เป็นการร่วงลงต่อเนื่องหลังจากเมื่อวันพฤหัสบดี(9ธ.ค.)บิตคอยน์ร่วงลงทะลุระดับ 50,000 ดอลลาร์ ท่ามกลางความกังวลเกี่ยวกับการที่ทางการสหรัฐจะเข้าควบคุมการซื้อขายสกุลเงินดิจิทัล

เมื่อเวลา 19.25 น.ตามเวลาไทย บิตคอยน์ดิ่งลง 2.73% สู่ระดับ 49,383.89 ดอลลาร์ในการซื้อขายบนแพลตฟอร์ม Coinbase

นักลงทุนกังวลว่า สหรัฐจะคุมเข้มการทำธุรกรรมสกุลเงินคริปโทฯ หลังจากที่สภาคองเกรสเรียกผู้บริหารของแพลตฟอร์มซื้อขายสกุลเงินดิจิทัลรายใหญ่ รวมทั้งผู้ที่เกี่ยวข้องกับธุรกิจขุดเหมืองบิตคอยน์เข้าให้การต่อคณะกรรมาธิการบริการการเงินประจำสภาผู้แทนราษฎรเมื่อวานนี้

ขณะนี้ทางการสหรัฐกำลังพิจารณาเกี่ยวกับการควบคุมธุรกรรมสกุลเงินดิจิทัล โดยอาจให้อยู่ภายใต้การกำกับดูแลของสำนักงานคณะกรรมการกำกับหลักทรัพย์และตลาดหลักทรัพย์ของสหรัฐ (กลต.) หรืออาจมีการตั้งหน่วยงานใหม่ขึ้นมากำหนดกฎระเบียบควบคุมธุรกิจดังกล่าว

นอกจากนี้ บิตคอยน์ยังถูกกดดันจากความกังวลที่ว่าสหรัฐอาจเรียกเก็บภาษีสกุลเงินคริปโทฯ

ทั้งนี้ นักลงทุนวิตกว่าสหรัฐจะทำก

In [98]:
# Save model
filename = 'model_ttvm2.joblib'
joblib.dump(model_m2, filename)

['model_ttvm2.joblib']

**2.4.3 Deep-learn model**

In [ ]:
# Bkkbiz 'deep-learn' model
model_b3 = Top2Vec(documents=bkkbiz_list, speed='deep-learn', tokenizer=tokenize)
b3_topic_words, b3_word_scores, b3_topic_nums = model_b3.get_topics()

In [ ]:
# Calling topic and related words from BangkokBiz corpus from 'learn' model
b3_topic_words[0:5]

In [ ]:
# Call topic related documents
top2vec_doc(model=model_b3, topic_num=1)

In [ ]:
# Save model
filename = 'model_ttvb3.joblib'
joblib.dump(model_b3, filename)

In [ ]:
# Matichon 'deep-learn' model
model_m3 = Top2Vec(documents=matichon_list, speed='learn', tokenizer=tokenize)
m3_topic_words, m3_word_scores, m3_topic_nums = model_m3.get_topics()

In [ ]:
# Calling topic and related words from BangkokBiz corpus from 'learn' model
m3_topic_words[0:20]

In [ ]:
# Call topic related documents
top2vec_doc(model=model_m3, topic_num=1)

In [ ]:
# Save model
filename = 'model_ttvm3.joblib'
joblib.dump(model_m3, filename)

# Part 3: Conclusion and Next Steps

**Conclusion**
- The Top2Vec models perform the best among 4 available choices.
- With Top2Vec, we are successfully filter related articles on the topics regarding Digital Competitiveness Ranking.
- We successfully map back the topics to each soft-data based indicators as follows:

|Domain|Indicator| Top2Vec | Topic |
|---|---|---|---|
|Knowledge| > Foreign highly-skilled personnel| | Education |
|  |> Digital/Technological skills | |Digital skills|
| |> Employee Training| 
| |> Knowledge Transfer|
| Law&Regulation| > Immigration Law|| Digital skills| 
|| > Scientific research legislation||Smart city
|| > Intellectual property rights| |Law and Crime|
|| > Cyber security| | AI|
|| > Development and Application of Technology||EEC
|Investment| > Venture capital| | Venture capital|
||> Funding of technological development||
||>Public-private partnerships
|Infrastructure|> Management of cities||Smart city|
||> Banking and financial services||Banking
||> Communication technology||Telecommunications
|Digital Economy|> Agility of companies||Venture capital
||> Opportunities and threats||AI
||> Use of big data and analytics||EEC
||> Internet experience

- These can be used as text filter for further policy analysis.
- This can help scan through a vast number of texts and thus saves time and increase higher text accuracy.

**Next-Steps**
- Try BERTopic model, where it helps find the most appropriate topic and documents by maximizing candidate relevance.
- This model requires a heavy research to understand how it was trained, how it performs and how can the parameters be tuned to fit with Thai language.

**Potential Use Case at True**
- Topic modeling with True's after-sales services data from call-center, non-voice services, IVR and Mari
- Potential impact:
    - Categorizing after-sales data in a more granular level
    - Reduce human errors during data input stage
    - Finding new-emerging after-sales service clusters